In [1]:
%cd '/content/drive/My Drive/ML/CS114.K21.KHTN/Final'

/content/drive/My Drive/ML/CS114.K21.KHTN/Final


#**6.Tuning hyperparameters**

##**6.1 Import các thư viện cần thiết**

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import joblib
import h5py
import numpy as np
import glob
import cv2

from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold

##**6.2 Tuning model SVM on normal vector** 

###**6.2.1 Load file data chứa các vector rút trích được lưu trữ trước đó**

In [ ]:
train_normal = 'H5PY/train/train_normal.h5'
train_label = 'H5PY/train/labels_train.h5'

In [ ]:
h5f_train = h5py.File(train_normal, 'r')
h5f_labels = h5py.File(train_label, 'r')

In [ ]:
data = h5f_train['dataset']
labels = h5f_labels['dataset']

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
h5f_train.close()
h5f_labels.close()

###**6.2.2 Chiều của dữ liệu và nhãn**

In [ ]:
data.shape

(13720, 3072)

In [ ]:
labels.shape

(13720,)

###**6.2.3 Khởi tạo model và cài đặt tham số cần tinh chỉnh**

In [ ]:
model = SVC()

In [ ]:
param_grid = {'C': [0.1, 1],  
              'gamma': [1, 0.1], 
              'kernel': ['linear', 'poly', 'rbf']}  

In [ ]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [ ]:
grid_random = RandomizedSearchCV(model, param_grid, cv=skf,n_jobs=-1,scoring='accuracy')

In [ ]:
grid_random.fit(data,labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [0.1, 1], 'gamma': [1, 0.1],
                                        'kernel': ['linear', 'poly', 'rbf']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

###**6.2.4 In ra kết quả bộ tham số tốt nhất của model và best_score đạt được**

In [ ]:
print(grid_random.best_estimator_)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
print(grid_random.best_score_)

0.9682944606413993


###**6.2.5 Lưu model sau khi tinh chỉnh**

In [ ]:
model_path = 'H5PY/model_tuning/svm_tuning_normal.sav'

In [ ]:
joblib.dump(grid_random.best_estimator_, model_path)

['H5PY/model_tuning/svm_tuning_normal.sav']

##**6.3 Tuning model Knn on local binary patterns vector** 

###**6.3.1 Load file data chứa các vector rút trích được lưu trữ trước đó**

In [ ]:
train_lbps = 'H5PY/train/train_lbps.h5'
train_label = 'H5PY/train/labels_train.h5'

In [ ]:
h5f_train = h5py.File(train_lbps, 'r')
h5f_labels = h5py.File(train_label, 'r')

In [ ]:
data = h5f_train['dataset']
labels = h5f_labels['dataset']

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
h5f_train.close()
h5f_labels.close()

###**6.3.2 Chiều của dữ liệu và nhãn**

In [ ]:
data.shape

(13720, 26)

In [ ]:
labels.shape

(13720,)

###**6.3.3 Khởi tạo model và cài đặt tham số cần tinh chỉnh**

In [ ]:
model = KNeighborsClassifier()

In [ ]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

In [ ]:
hyperparameters = dict(leaf_size=leaf_size,n_neighbors=n_neighbors,p=p)

In [ ]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [ ]:
grid_random = RandomizedSearchCV(model, hyperparameters, cv=skf,n_jobs=-1,scoring='accuracy')

In [ ]:
grid_random.fit(data,labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
           

###**6.3.4 In ra kết quả bộ tham số tốt nhất của model và best_score đạt được**

In [ ]:
print(grid_random.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=18, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=1,
                     weights='uniform')


In [ ]:
print(grid_random.best_score_)

0.7392128279883382


###**6.3.5 Lưu model sau khi tinh chỉnh**

In [ ]:
model_path = 'H5PY/model_tuning/knn_tuning_lbps.sav'

In [ ]:
joblib.dump(grid_random.best_estimator_, model_path)

['H5PY/model_tuning/knn_tuning_lbps.sav']

##**6.4 Tuning model SVM on histogram vector** 

###**6.4.1 Load file data chứa các vector rút trích được lưu trữ trước đó**

In [3]:
train_hog = 'H5PY/train/train_hog.h5'
train_label = 'H5PY/train/labels_train.h5'

In [4]:
h5f_train = h5py.File(train_hog, 'r')
h5f_labels = h5py.File(train_label, 'r')

In [5]:
data = h5f_train['dataset']
labels = h5f_labels['dataset']

In [6]:
data = np.array(data)
labels = np.array(labels)

In [7]:
h5f_train.close()
h5f_labels.close()

###**6.4.2 Chiều của dữ liệu và nhãn**

In [8]:
data.shape

(13720, 144)

In [9]:
labels.shape

(13720,)

###**6.2.3 Khởi tạo model và cài đặt tham số cần tinh chỉnh**

In [11]:
model = SVC()

In [10]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  

In [12]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [13]:
grid_random = RandomizedSearchCV(model, param_grid, cv=skf,n_jobs=-1,scoring='accuracy')

In [14]:
grid_random.fit(data,labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return

###**6.4.4 In ra kết quả bộ tham số tốt nhất của model và best_score đạt được**

In [15]:
print(grid_random.best_estimator_)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [16]:
print(grid_random.best_score_)

0.822521865889213


###**6.4.5 Lưu model sau khi tinh chỉnh**

In [17]:
model_path = 'H5PY/model_tuning/svm_tuning_hog.sav'

In [18]:
joblib.dump(grid_random.best_estimator_, model_path)

['H5PY/model_tuning/svm_tuning_hog.sav']